<a href="https://colab.research.google.com/github/ohibka/rty/blob/main/chatBotpizza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
python
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, ConversationHandler
from telegram import ParseMode, ReplyKeyboardMarkup, ReplyKeyboardRemove, InlineKeyboardButton, InlineKeyboardMarkup

# Global variables
pizzas = {
    'margarita': {'name': '🍅 Маргарита', 'price': 100},
    'pepperoni': {'name': '🍕 Пепперони', 'price': 150},
    'hawaiian': {'name': '🍍 Гавайская', 'price': 200},
    'vegetarian': {'name': '🥦 Вегетарианская', 'price': 180},
}

sizes = {'small': {'name': '🔹 Маленькая', 'price_multiplier': 1},
    'medium': {'name': '🔸 Средняя', 'price_multiplier': 1.5},
    'large': {'name': '🔶 Большая', 'price_multiplier': 2},
}

# Steps
SELECTING_PIZZA, SELECTING_SIZE, ENTERING_ADDRESS = range(3)

# Handler functions
def start(update, context):
    user = update.effective_user
    context.bot.send_message(chat_id=user.id, text=f'Здравствуйте, {user.first_name}! Выберите пиццу из списка.')

    pizza_buttons = [InlineKeyboardButton(pizza['name'], callback_data=f'pizza_{pizza_id}')
                     for pizza_id, pizza in pizzas.items()]
    pizza_markup = InlineKeyboardMarkup.from_column(pizza_buttons)

    context.user_data.clear()
    context.user_data['pizza_id'] = None
    context.user_data['size_id'] = None
    update.message.reply_text(text='Выберите пиццу:', reply_markup=pizza_markup)
    return SELECTING_PIZZA

def select_pizza(update, context):
    query = update.callback_query
    pizza_id = query.data.split('_')[-1]
    context.user_data['pizza_id'] = pizza_id
    query.edit_message_text(text='Выберите размер пиццы')

    size_buttons = [InlineKeyboardButton(size['name'], callback_data=f'size_{size_id}')
                    for size_id, size in sizes.items()]
    size_markup = InlineKeyboardMarkup.from_column(size_buttons)

    return SELECTING_SIZE

def select_size(update, context):
    query = update.callback_query
    size_id = query.data.split('_')[-1]
    context.user_data['size_id'] = size_id

    query.edit_message_text(
        text='Введите адрес доставки',
        reply_markup=ReplyKeyboardMarkup([['Отменить']], one_time_keyboard=True),
    )

    return ENTERING_ADDRESS

def enter_address(update, context):
    user_input = update.message.text
    chat_id = update.message.chat_id
    if user_input == 'Отменить':
        context.bot.send_message(chat_id=chat_id, text='Заказ отменен.')
        return ConversationHandler.END

    pizza_id = context.user_data['pizza_id']
    size_id = context.user_data['size_id']

    invalid_address_message = 'Адрес доставки введен не корректно. Введите адрес еще раз.'
    if len(user_input) <= 5:
        context.bot.send_message(chat_id=chat_id, text=invalid_address_message)
        return ENTERING_ADDRESS

    pizza = pizzas.get(pizza_id)
    size = sizes.get(size_id)
    price = pizza.get('price', 0) * size.get('price_multiplier', 1)

    message_text = f'Спасибо за ваш заказ: {pizza.get("name")}, {size.get("name")} размер!\nАдрес доставки: {user_input}\nЦена: {price} руб.'
    context.bot.send_message(chat_id=chat_id, text=message_text)

    context.user_data.clear()
    return ConversationHandler.END

def cancel_order(update, context):
    chat_id = update.message.chat_id
    context.bot.send_message(chat_id=chat_id, text='Заказ отменен.')
    context.user_data.clear()
    return ConversationHandler.END

def main():
    updater = Updater(token='TOKEN', use_context=True)

    reorder_keyboard = ReplyKeyboardMarkup([['🔁 Повторить заказ']])
    convo_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            SELECTING_PIZZA: [CallbackQueryHandler(select_pizza, pattern='^pizza_')],
            SELECTING_SIZE: [CallbackQueryHandler(select_size, pattern='^size_')],
            ENTERING_ADDRESS: [
                MessageHandler(Filters.text, enter_address),
                MessageHandler(Filters.regex('^Отменить$'), cancel_order),
            ]
        },
        fallbacks=[MessageHandler(Filters.regex('^🔁 Повторить заказ$'), start)],
allow_reentry=True
    )

    updater.dispatcher.add_handler(convo_handler)
    updater.start_polling()

if __name__ == '__main__':
    main()


NameError: ignored